## Using SQL Express Developer Edition on local laptop

**Stage 1, import original selections**

In [ ]:
IF OBJECT_ID ('TEMPDB..#ORIGINAL_SELECTION') IS NOT NULL DROP TABLE #ORIGINAL_SELECTION
GO
CREATE TABLE #ORIGINAL_SELECTION
(	
	CUSTOMER_ID int,
	LINK VARCHAR(250)
);

BULK INSERT #ORIGINAL_SELECTION
FROM 'C:\Users\napl\Downloads\customer_ids_links.csv'
WITH (
	DATAFILETYPE = 'char',
	FIRSTROW = 2,
	FIELDTERMINATOR = ',',
	ROWTERMINATOR = '\n' );

SELECT TOP 10 * FROM #ORIGINAL_SELECTION

**We need the value from the LINK column where the key is RID and store it in the same table so we can compare.**

In [ ]:
ALTER TABLE #ORIGINAL_SELECTION ADD RID BIGINT;
GO
UPDATE #ORIGINAL_SELECTION SET RID = REPLACE(LINK, 'https://bb.survey.com/G1122NL/survey/my_survey?name=G1122NL&RID=','');
GO
SELECT TOP 10 * FROM #ORIGINAL_SELECTION

**Import winners file**

In [ ]:
IF OBJECT_ID ('TEMPDB..#WINNERS') IS NOT NULL DROP TABLE #WINNERS
CREATE TABLE #WINNERS (
	[ID]	BIGINT
);
GO
BULK INSERT #WINNERS
FROM 'C:\Users\napl\Downloads\winners.csv'
WITH (
	DATAFILETYPE = 'char',
	FIRSTROW = 2,
	FIELDTERMINATOR = ',',
	ROWTERMINATOR = '\n' );
GO
SELECT TOP 10 * FROM #WINNERS;

**Check winners ID in relation to the original selection, this selection reveals the winners from the selection**

**Combine into an new temporary table, assign an identity column that we can use later..**

In [8]:
IF OBJECT_ID ('TEMPDB..#EXPORT_WINNERS') IS NOT NULL DROP TABLE #EXPORT_WINNERS
GO
CREATE TABLE #EXPORT_WINNERS (
    ID          INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
    CUSTOMER_ID INT,
    CARDNUMBER  BIGINT NULL,
    PIN VARCHAR(4)
);
GO
INSERT INTO #EXPORT_WINNERS (CUSTOMER_ID)
SELECT  S.CUSTOMER_ID
FROM    #WINNERS W
JOIN    #ORIGINAL_SELECTION S ON W.ID = S.RID;
GO
SELECT TOP 10 * FROM #EXPORT_WINNERS

Commands completed successfully.

Commands completed successfully.

(150 rows affected)

(10 rows affected)

Total execution time: 00:00:00.036

ID,CUSTOMER_ID,CARDNUMBER,PIN
1,4214941,NULL,NULL
2,4276878,NULL,NULL
3,4170876,NULL,NULL
4,4207486,NULL,NULL
5,4194813,NULL,NULL
6,4033599,NULL,NULL
7,3938711,NULL,NULL
8,4252354,NULL,NULL
9,4017893,NULL,NULL
10,4121731,NULL,NULL


**Import giftcard file**

In [ ]:
IF OBJECT_ID ('TEMPDB..#GIFTCARDS') IS NOT NULL DROP TABLE #GIFTCARDS
CREATE TABLE #GIFTCARDS
(	
	
	CARDNUMBER  BIGINT,
	PIN         VARCHAR(20),
	VALUE       INT
);
GO
BULK INSERT #GIFTCARDS
FROM 'C:\Users\napl\Downloads\giftcards.csv'
WITH (
    DATAFILETYPE = 'char',
    FIRSTROW = 2,
    FIELDTERMINATOR = ',',
    ROWTERMINATOR = '\n' );
GO
SELECT TOP 10 * FROM #GIFTCARDS

**Add identity column to the giftcards table, from 1 tom 150.... with that we have an relational key with the export\_winners table.**

In [9]:
ALTER TABLE #GIFTCARDS ADD ROWID INT IDENTITY(1,1) NOT NULL PRIMARY KEY;
GO

SELECT TOP 10 * FROM #GIFTCARDS

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.049

CARDNUMBER,PIN,VALUE,ROWID
174943770439,9777,5,1
197827564076,6563,5,2
209293460449,4580,5,3
188081367736,6594,5,4
191237122026,2238,5,5
125154336924,4602,5,6
168101825465,5861,5,7
237409550065,7893,5,8
179823004372,2868,5,9
157866566156,7702,5,10


**Now we can join the winners and the giftcards to form 1 table, we update the corresponding values**

**Then every customer has been assigned an unique cardnumber and pincode**

In [10]:
SELECT *
FROM    #EXPORT_WINNERS E
JOIN    #GIFTCARDS G ON E.ID = G.ROWID

GO

(150 rows affected)

Total execution time: 00:00:00.021

ID,CUSTOMER_ID,CARDNUMBER,PIN,CARDNUMBER,PIN,VALUE,ROWID
1,4214941,NULL,NULL,174943770439,9777,5,1
2,4276878,NULL,NULL,197827564076,6563,5,2
3,4170876,NULL,NULL,209293460449,4580,5,3
4,4207486,NULL,NULL,188081367736,6594,5,4
5,4194813,NULL,NULL,191237122026,2238,5,5
6,4033599,NULL,NULL,125154336924,4602,5,6
7,3938711,NULL,NULL,168101825465,5861,5,7
8,4252354,NULL,NULL,237409550065,7893,5,8
9,4017893,NULL,NULL,179823004372,2868,5,9
10,4121731,NULL,NULL,157866566156,7702,5,10


In [12]:
-- MAKE IT FINAL
UPDATE E 
    SET  CARDNUMBER = G.CARDNUMBER,
        PIN = G.PIN
FROM    #EXPORT_WINNERS E
JOIN    #GIFTCARDS G ON E.ID = G.ROWID
GO

(150 rows affected)

Total execution time: 00:00:00.004

**Get the final table**

In [14]:
SELECT CUSTOMER_ID, CARDNUMBER, PIN
FROM #EXPORT_WINNERS

(150 rows affected)

Total execution time: 00:00:00.016

CUSTOMER_ID,CARDNUMBER,PIN
4214941,174943770439,9777
4276878,197827564076,6563
4170876,209293460449,4580
4207486,188081367736,6594
4194813,191237122026,2238
4033599,125154336924,4602
3938711,168101825465,5861
4252354,237409550065,7893
4017893,179823004372,2868
4121731,157866566156,7702
